In [2]:
import pandas as pd

# SD Cosyne Img Reward

In [3]:
sd_cosyne_img_reward = pd.read_csv("results/evals/sd_cosyne_img_reward/measurements.csv")
sd_cosyne_img_reward['name'] = sd_cosyne_img_reward['img_path'].apply(lambda x: x.split('/')[-1].split('.')[0])

In [4]:
filtered_df = sd_cosyne_img_reward[~sd_cosyne_img_reward['name'].isin(['max', 'baseline'])].copy()
filtered_df['name'] = filtered_df['name'].astype(int)
filtered_df = filtered_df.loc[filtered_df['name'].isin(range(17))] # ONLY LOOK AT THE FIRST 16 IMAGES.
filtered_df = filtered_df.sort_values(by='name')

In [5]:
data = []
for prompt, group in filtered_df.groupby('prompt'):
    zero_row = group.loc[group['name'] == 0]
    max_row = group.loc[group['img_reward_score'].idxmax()]
    delta_img_reward = (max_row['img_reward_score'] - zero_row['img_reward_score']).item()
    delta_hpsv_reward = (max_row['hpsv_reward_score'] - zero_row['hpsv_reward_score']).item()
    delta_aesthetic_reward = (max_row['aesthetic_reward_score'] - zero_row['aesthetic_reward_score']).item()
    delta_clip_reward = (max_row['clip_reward_score'] - zero_row['clip_reward_score']).item()
    data.append({
        'prompt': prompt,
        'delta_img_reward': delta_img_reward,
        'delta_clip_reward': delta_clip_reward,
        'delta_aesthetic_reward': delta_aesthetic_reward,
        'delta_hpsv_reward': delta_hpsv_reward,
    })

In [6]:
delta_data = pd.DataFrame(data)
delta_data.describe()


,delta_img_reward,delta_clip_reward,delta_aesthetic_reward,delta_hpsv_reward
count,200.000000,198.000000,200.000000,200.000000
mean,1.582081,2.885195,0.061938,0.012862
std,1.023972,4.373580,0.446643,0.013101
min,0.084769,-10.473164,-1.193415,-0.019300
25%,0.760760,0.026792,-0.250453,0.004125
50%,1.383070,1.969116,0.103652,0.012200
75%,2.244525,5.596532,0.323349,0.020500
max,4.093665,17.514080,1.332358,0.053000


# SD SNES Img Reward

In [49]:
sd_snes_img_reward = pd.read_csv("results/evals/sd_snes_img_reward/measurements.csv")
sd_snes_img_reward['name'] = sd_snes_img_reward['img_path'].apply(lambda x: x.split('/')[-1].split('.')[0])

In [ ]:
filtered_df = sd_cosyne_img_reward[~sd_snes_img_reward['name'].isin(['max', 'baseline'])].copy()
filtered_df['name'] = filtered_df['name'].astype(int)
filtered_df = filtered_df.loc[filtered_df['name'].isin(range(17))] # ONLY LOOK AT THE FIRST 16 IMAGES.
filtered_df = filtered_df.sort_values(by='name')

In [ ]:
data = []
for prompt, group in sd_snes_img_reward.groupby('prompt'):
    zero_row = group.loc[group['name'] == 0]
    max_row = group.loc[group['img_reward_score'].idxmax()]
    delta_img_reward = (max_row['img_reward_score'] - zero_row['img_reward_score']).item()
    delta_hpsv_reward = (max_row['hpsv_reward_score'] - zero_row['hpsv_reward_score']).item()
    delta_aesthetic_reward = (max_row['aesthetic_reward_score'] - zero_row['aesthetic_reward_score']).item()
    delta_clip_reward = (max_row['clip_reward_score'] - zero_row['clip_reward_score']).item()
    data.append({
        'prompt': prompt,
        'delta_img_reward': delta_img_reward,
        'delta_clip_reward': delta_clip_reward,
        'delta_aesthetic_reward': delta_aesthetic_reward,
        'delta_hpsv_reward': delta_hpsv_reward,
    })

In [51]:
delta_data = pd.DataFrame(data)
delta_data.describe()


,delta_img_reward,delta_clip_reward,delta_aesthetic_reward,delta_hpsv_reward
count,200.000000,198.000000,200.000000,200.000000
mean,1.475520,2.603947,0.049604,0.009237
std,0.967118,4.202388,0.404115,0.013002
min,0.090222,-5.922293,-1.225650,-0.019000
25%,0.709093,-0.447540,-0.223964,0.000300
50%,1.316835,1.943671,0.073169,0.007950
75%,2.071716,5.276017,0.317035,0.016275
max,4.090756,17.252880,1.047339,0.051000


In [7]:
from eval_datasets import create_dataset
from omegaconf import OmegaConf, DictConfig
import pandas as pd

In [16]:
pd.read_csv("eval_datasets/open_img_pref/open_img_pref_all.csv").groupby('category').sample(n=3).to_csv("eval_datasets/open_img_pref/open_img_pref_sampled_36.csv", index=False)

In [4]:
cfg = DictConfig({"name": "open_image_preferences"})
dataset = create_dataset(cfg)